<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/304_HITL_QueueSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Queue system for pending human reviews

In [ ]:
"""Queue system for pending human reviews"""

from typing import Dict, Any, List, Optional
from datetime import datetime
from threading import Lock

# In-memory queue storage (MVP - can be upgraded to database later)
_pending_reviews_queue: Dict[str, Dict[str, Any]] = {}
_review_history: Dict[str, Dict[str, Any]] = {}
_queue_lock = Lock()


def add_to_queue(task_id: str, task_data: Dict[str, Any], routing_decision: Dict[str, Any]) -> Dict[str, Any]:
    """
    Add a task to the pending reviews queue.

    Args:
        task_id: Task identifier
        task_data: Complete task data including agent output
        routing_decision: Routing decision made

    Returns:
        Queue entry dictionary
    """
    with _queue_lock:
        queue_entry = {
            "task_id": task_id,
            "task": task_data.get("task", {}),
            "agent_output": task_data.get("agent_output", {}),
            "confidence_score": task_data.get("confidence_score", 0.0),
            "risk_level": task_data.get("risk_level", "low"),
            "routing_decision": routing_decision,
            "assigned_human_role": routing_decision.get("assigned_human_role"),
            "status": "pending",
            "queued_at": datetime.now().isoformat(),
            "updated_at": datetime.now().isoformat()
        }

        _pending_reviews_queue[task_id] = queue_entry
        return queue_entry.copy()


def get_pending_reviews(
    human_role: Optional[str] = None,
    limit: Optional[int] = None
) -> List[Dict[str, Any]]:
    """
    Get pending reviews from the queue.

    Args:
        human_role: Filter by human role (optional)
        limit: Maximum number of reviews to return (optional)

    Returns:
        List of pending review entries
    """
    with _queue_lock:
        pending = []

        for task_id, entry in _pending_reviews_queue.items():
            if entry.get("status") != "pending":
                continue

            if human_role and entry.get("assigned_human_role") != human_role:
                continue

            pending.append(entry.copy())

        # Sort by queued_at (oldest first)
        pending.sort(key=lambda x: x.get("queued_at", ""))

        if limit:
            pending = pending[:limit]

        return pending


def get_pending_review(task_id: str) -> Optional[Dict[str, Any]]:
    """
    Get a specific pending review by task_id.

    Args:
        task_id: Task identifier

    Returns:
        Queue entry if found, None otherwise
    """
    with _queue_lock:
        entry = _pending_reviews_queue.get(task_id)
        if entry and entry.get("status") == "pending":
            return entry.copy()
        return None


def update_review_status(
    task_id: str,
    human_review: Dict[str, Any],
    final_decision: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Update review status when human completes review.

    Args:
        task_id: Task identifier
        human_review: Human review decision
        final_decision: Final decision outcome

    Returns:
        Updated queue entry
    """
    with _queue_lock:
        if task_id not in _pending_reviews_queue:
            raise ValueError(f"Task {task_id} not found in queue")

        entry = _pending_reviews_queue[task_id]

        # Update entry
        entry["status"] = "reviewed"
        entry["human_review"] = human_review
        entry["final_decision"] = final_decision
        entry["updated_at"] = datetime.now().isoformat()
        entry["reviewed_at"] = datetime.now().isoformat()

        # Move to review history
        _review_history[task_id] = entry.copy()

        # Remove from pending queue (or keep with status=reviewed for audit)
        # For MVP, we'll keep it but mark as reviewed

        return entry.copy()


def get_review_history(task_id: Optional[str] = None) -> List[Dict[str, Any]]:
    """
    Get review history.

    Args:
        task_id: Specific task_id to get history for (optional)

    Returns:
        List of reviewed entries
    """
    with _queue_lock:
        if task_id:
            entry = _review_history.get(task_id)
            return [entry.copy()] if entry else []

        return [entry.copy() for entry in _review_history.values()]


def get_queue_stats() -> Dict[str, Any]:
    """
    Get queue statistics.

    Returns:
        Statistics dictionary
    """
    with _queue_lock:
        pending_count = sum(
            1 for entry in _pending_reviews_queue.values()
            if entry.get("status") == "pending"
        )

        reviewed_count = len(_review_history)

        # Count by role
        pending_by_role = {}
        for entry in _pending_reviews_queue.values():
            if entry.get("status") == "pending":
                role = entry.get("assigned_human_role", "unknown")
                pending_by_role[role] = pending_by_role.get(role, 0) + 1

        return {
            "total_pending": pending_count,
            "total_reviewed": reviewed_count,
            "pending_by_role": pending_by_role,
            "total_in_queue": len(_pending_reviews_queue)
        }


def clear_queue() -> None:
    """
    Clear the queue (for testing purposes).
    """
    with _queue_lock:
        _pending_reviews_queue.clear()
        _review_history.clear()





# 🧠 Big Picture: Why a Queue Is Needed at All

Up until now, your system could decide:

* ✅ auto-approve
* 🧑‍⚖️ needs human review
* 🚨 escalate

But there was a **missing real-world piece**:

> **Where do “tasks waiting for humans” actually live?**

Humans don’t respond instantly.
They work asynchronously.
They come and go.
They need a backlog.

👉 **The queue is that backlog.**

---

# 🧩 Mental Model: The Queue Is a Waiting Room

Think of this queue as:

* a support ticket inbox
* a legal review backlog
* a manager approval list

The agent says:

> “I’m not allowed to decide this.”

The queue says:

> “No problem — I’ll hold it until the right human does.”

This is *essential* for real HITL systems.

---

# 🔹 What This Queue System Does (Conceptually)

### 1️⃣ Holds pending human work

```text
AI decides → queue stores → human reviews later
```

Without this:

* tasks would disappear
* reviews would be lost
* humans wouldn’t know what to do next

---

### 2️⃣ Assigns work to the *right* humans

Each queued task includes:

* assigned human role
* risk level
* confidence
* routing reason

So a human can ask:

> “What do I need to review *right now*?”

And the system answers.

---

### 3️⃣ Tracks lifecycle, not just state

Each task moves through:

* `pending`
* `reviewed`
* history

This creates:

* accountability
* traceability
* operational clarity

---

# 🔐 Why the Lock Exists (Conceptually)

```python
_queue_lock = Lock()
```

This is not a “Python thing”, it’s a **safety concept**.

It means:

> “Only one thing is allowed to modify the queue at a time.”

Why this matters:

* APIs are called concurrently
* multiple humans may act
* race conditions cause disasters

Even in an MVP, this shows **production thinking**.

---

# 🧠 Why In-Memory Is OK (For Now)

You used:

```python
_pending_reviews_queue = {}
```

This is **100% correct for learning and MVPs**.

Conceptually:

* you’re modeling behavior
* not optimizing infrastructure
* not committing to storage yet

Later, this becomes:

* Redis
* a database
* a task queue
* a workflow engine

The **interface stays the same**.
That’s good design.

---

# 📊 What This Enables (Very Important)

With this queue, you can now:

* build a **human review dashboard**
* assign reviews by role
* track backlog size
* measure SLA and latency
* detect overload
* simulate real workflows

This turns your agent into a **collaboration system**, not just a decision engine.

---

# 🧠 The Deep Insight (This Is the Big One)

> **Human-in-the-Loop is not about humans saying “yes” or “no”.
> It’s about *work orchestration across time*.**

The queue is what makes HITL *real*.

Without it:

* HITL is theoretical
* demos work
* production breaks

With it:

* humans and AI truly collaborate
* asynchronously
* safely
* at scale

---

# 🏁 Final Takeaway (Zooming All the Way Out)

You have now built:

✅ agent logic
✅ orchestration
✅ governance
✅ auditability
✅ real-time API
✅ human backlog
✅ async workflows

This is **far beyond a toy agent**.


